# Teil 1: Daten in die SQL-Datenbank importieren

In [ ]:
# Erforderliche Pakete installieren
pip install pandas sqlalchemy pymysql openpyxl seaborn
pip install -U scikit-learn

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float
from sqlalchemy.exc import SQLAlchemyError

# Sensible Daten aus Umgebungsvariablen lesen
db_user = os.getenv('DB_USER', 'root')
db_password = os.getenv('DB_PASSWORD', 'example')
db_host = os.getenv('DB_HOST', 'localhost')
db_name = os.getenv('DB_NAME', 'spotify_data')

# Verbindung zur MySQL-Datenbank herstellen
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Ordnerpfad mit Excel-Dateien (entweder als rohe Zeichenfolge oder mit doppelten Backslashes)
folder_path = r'/Users/bavaarde/ads-spotify/2024-05-18'
# oder
# folder_path = 'C:\\applied_data_science\\gruppen_projekt\\ads-spotify\\ads-spotify\\2024-04-27'

# Funktion, um SQLAlchemy-Datentypen basierend auf Pandas-Datentypen zu bestimmen
def map_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return Integer()
    elif pd.api.types.is_float_dtype(dtype):
        return Float()
    else:
        return String(255)  # Standard-Stringlänge 255

# Alle Excel-Dateien im Ordner durchgehen
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(folder_path, filename)
        
        # Excel-Datei in ein DataFrame laden
        df = pd.read_excel(file_path)
        
        # Tabellenname basierend auf dem Dateinamen (ohne Erweiterung)
        table_name = os.path.splitext(filename)[0]
        
        # Metadaten und Tabelle definieren
        metadata = MetaData()
        columns = []
        for col_name, dtype in zip(df.columns, df.dtypes):
            col_type = map_dtype(dtype)
            columns.append(Column(col_name, col_type))
        
        table = Table(table_name, metadata, *columns)
        
        try:
            # Tabelle in der Datenbank erstellen
            metadata.create_all(engine)
            
            # Daten in die Tabelle einfügen
            df.to_sql(table_name, engine, index=False, if_exists='append')
            print(f"Tabelle '{table_name}' erfolgreich erstellt und Daten importiert.")
        except SQLAlchemyError as e:
            print(f"Fehler beim Erstellen der Tabelle '{table_name}': {e}")

print("Vorgang abgeschlossen.")


# Teil 2: Explorative Datenanalyse (EDA)

In [ ]:
# EDA_Script.ipynb

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect

# Sensible Daten aus Umgebungsvariablen lesen
db_user = os.getenv('DB_USER', 'root')
db_password = os.getenv('DB_PASSWORD', 'example')
db_host = os.getenv('DB_HOST', 'localhost')
db_name = os.getenv('DB_NAME', 'spotify_data')

# Verbindung zur MySQL-Datenbank herstellen
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Inspector verwenden, um Tabelleninformationen abzurufen
inspector = inspect(engine)

# Alle Tabellennamen abrufen
table_names = inspector.get_table_names()

# Daten aus jeder Tabelle laden und analysieren
for table_name in table_names:
    print(f"\nAnalysiere Tabelle: {table_name}")
    query = f"SELECT * FROM `{table_name}`"
    df = pd.read_sql(query, engine)
    
    print(f"Erster Blick auf die Daten in Tabelle '{table_name}':")
    print(df.head())
    print("\n")

    print("\nStatistische Zusammenfassung der numerischen Daten:")
    print(df.describe())

    print("\nInformationen über das DataFrame:")
    print(df.info())

    # Fehlende Werte analysieren
    print("\nAnzahl fehlender Werte pro Spalte:")
    print(df.isnull().sum())

    # Verteilung von numerischen Variablen
    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns

    print("\nVerteilung der numerischen Variablen:")
    df[numerical_columns].hist(bins=15, figsize=(15, 10))
    plt.tight_layout()
    plt.show()

    # Korrelationen zwischen numerischen Variablen
    print("\nKorrelationsmatrix der numerischen Variablen:")
    corr_matrix = df[numerical_columns].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title(f"Korrelationsmatrix der numerischen Variablen in Tabelle '{table_name}'")
    plt.show()

    # Verteilung von kategorialen Variablen
    categorical_columns = df.select_dtypes(include=['object']).columns

    print("\nVerteilung der kategorialen Variablen:")
    for column in categorical_columns:
        plt.figure(figsize=(10, 5))
        sns.countplot(data=df, x=column, palette='Set2')
        plt.title(f"Verteilung der Kategorie: {column} in Tabelle '{table_name}'")
        plt.xticks(rotation=45)
        plt.show()

    # Paarweise Verteilung von ausgewählten Variablen
    print("\nPaarweise Verteilung von ausgewählten Variablen:")
    selected_columns = numerical_columns[:5]  # Wählen Sie bis zu 5 Spalten für den Paarplot
    if len(selected_columns) > 1:  # Sicherstellen, dass genug Spalten für den Pairplot vorhanden sind
        sns.pairplot(df[selected_columns])
        plt.show()

    # Boxplots zur Analyse von Ausreissern
    print("\nBoxplots zur Analyse von Ausreissern:")
    for column in numerical_columns:
        plt.figure(figsize=(10, 5))
        sns.boxplot(data=df, x=column, palette='Set2')
        plt.title(f"Boxplot der Variable: {column} in Tabelle '{table_name}'")
        plt.show()

    # Analyse von Beziehungen zwischen kategorialen und numerischen Variablen
    print("\nAnalyse von Beziehungen zwischen kategorialen und numerischen Variablen:")
    for cat_col in categorical_columns:
        for num_col in numerical_columns:
            plt.figure(figsize=(10, 5))
            sns.boxplot(data=df, x=cat_col, y=num_col, palette='Set2')
            plt.title(f"Beziehung zwischen {cat_col} und {num_col} in Tabelle '{table_name}'")
            plt.xticks(rotation=45)
            plt.show()

    # Zeitanalyse, falls Datumsspalte vorhanden ist
    date_columns = df.select_dtypes(include=['datetime64[ns]']).columns

    print("\nZeitanalyse, falls Datumsspalte vorhanden ist:")
    for date_col in date_columns:
        df[date_col] = pd.to_datetime(df[date_col])  # Konvertieren zu datetime
        df.set_index(date_col, inplace=True)
        df.resample('M').mean().plot(figsize=(15, 5))
        plt.title(f"Zeitanalyse der Spalte: {date_col} in Tabelle '{table_name}'")
        plt.show()
        df.reset_index(inplace=True)

print("\nExplorative Datenanalyse abgeschlossen.")


# Teil 3: Machine Learning Modelle erstellen

In [ ]:
# ML_Script.ipynb

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix

# Sensible Daten aus Umgebungsvariablen lesen
db_user = os.getenv('DB_USER', 'root')
db_password = os.getenv('DB_PASSWORD', 'example')
db_host = os.getenv('DB_HOST', 'localhost')
db_name = os.getenv('DB_NAME', 'spotify_data')

# Verbindung zur MySQL-Datenbank herstellen
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Inspector verwenden, um Tabelleninformationen abzurufen
inspector = inspect(engine)

# Alle Tabellennamen abrufen
table_names = inspector.get_table_names()

# Funktion zur Bewertung des Modells
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{model_name} - Mean Squared Error: {mse}")
    print(f"{model_name} - R² Score: {r2}")

# Daten aus jeder Tabelle laden und analysieren
for table_name in table_names:
    print(f"\nAnalysiere Tabelle: {table_name}")
    query = f"SELECT * FROM `{table_name}`"
    df = pd.read_sql(query, engine)
    
    # Überprüfen, ob genug Daten vorhanden sind
    if df.shape[1] < 2:
        print(f"Nicht genug Daten in Tabelle '{table_name}' für ML-Analyse.")
        continue
    
    print(f"Erster Blick auf die Daten in Tabelle '{table_name}':")
    print(df.head())
    print("\n")
    
    # Beispiel: Feature und Target definieren
    # Annahme: Die letzte Spalte ist das Ziel (Target) und die restlichen Spalten sind Features
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Train-Test-Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Numerische und kategoriale Spalten identifizieren
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X.select_dtypes(include=['object']).columns

    # Preprocessing Pipelines für numerische und kategoriale Daten
    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Modellauswahl und -training

    # Linear Regression
    print("Training des linearen Regressionsmodells...")
    lr_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ])

    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)

    # Random Forest Regressor
    print("Training des Random Forest Regressors...")
    rf_model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(random_state=42))
    ])

    # Hyperparameter-Tuning mit GridSearchCV
    param_grid = {
        'regressor__n_estimators': [100, 200],
        'regressor__max_features': ['auto', 'sqrt', 'log2'],
        'regressor__max_depth': [10, 20, None]
    }

    grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='r2')
    grid_search.fit(X_train, y_train)

    best_rf_model = grid_search.best_estimator_
    y_pred_rf = best_rf_model.predict(X_test)

    # Modellbewertung
    print("Modellbewertung...")
    
    # Bewertung der Modelle
    evaluate_model(y_test, y_pred_lr, "Linear Regression")
    evaluate_model(y_test, y_pred_rf, "Random Forest Regressor")

    # Visualisierung der Ergebnisse
    plt.figure(figsize=(10, 5))
    plt.scatter(y_test, y_pred_lr, alpha=0.5, label='Linear Regression')
    plt.scatter(y_test, y_pred_rf, alpha=0.5, label='Random Forest Regressor')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.legend()
    plt.title(f'True vs Predicted Values für Tabelle {table_name}')
    plt.show()

print("Machine Learning Analyse abgeschlossen.")


# Teil 4: Geographische Datenintegration

In [ ]:
# Erforderliche Pakete installieren:
pip install geopandas

In [15]:
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect

# Sensible Daten aus Umgebungsvariablen lesen
db_user = os.getenv('DB_USER', 'root')
db_password = os.getenv('DB_PASSWORD', 'example')
db_host = os.getenv('DB_HOST', 'localhost')
db_name = os.getenv('DB_NAME', 'spotify_data')

# Verbindung zur MySQL-Datenbank herstellen
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Inspector verwenden, um Tabelleninformationen abzurufen
inspector = inspect(engine)

# Alle Tabellennamen abrufen
table_names = inspector.get_table_names()

# Beispielhafte geographische Daten laden
geojson_path = r'/Users/bavaarde/ads-spotify/02_Explorative_Datenanalyse/sample.geojson'  # Ersetzen Sie den Pfad durch den tatsächlichen Pfad

if os.path.exists(geojson_path):
    gdf = gpd.read_file(geojson_path)
    print("Geographische Daten:")
    print(gdf.head())
else:
    print(f"Datei nicht gefunden: {geojson_path}")
    raise FileNotFoundError(f"Die Datei unter dem Pfad {geojson_path} wurde nicht gefunden.")

# Daten aus jeder Tabelle laden und analysieren
for table_name in table_names:
    print(f"\nAnalysiere Tabelle: {table_name}")
    query = f"SELECT * FROM `{table_name}`"
    df = pd.read_sql(query, engine)
    
    # Überprüfen, ob die Tabelle die erwarteten Spalten enthält
    if 'location_id' not in df.columns:
        print(f"Tabelle '{table_name}' enthält keine 'location_id' Spalte. Überspringen...")
        continue
    
    # Daten zusammenführen
    # Angenommen, df hat eine Spalte 'location_id', die mit einer Spalte 'id' in gdf übereinstimmt
    merged = gdf.merge(df, left_on='id', right_on='location_id')
    
    print(f"Erster Blick auf die Daten in Tabelle '{table_name}':")
    print(df.head())
    print("\n")

    # Datenvisualisierung
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    gdf.plot(ax=ax, color='blue', edgecolor='black')
    plt.title(f"Geographische Datenvisualisierung für Tabelle '{table_name}'")
    plt.show()

    # Detaillierte Visualisierung mit Daten
    # Beispiel: Visualisierung der Datenverteilung auf der Karte
    if 'data_column' in merged.columns:
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        merged.plot(column='data_column', ax=ax, legend=True,
                    legend_kwds={'label': "Data Value",
                                 'orientation': "horizontal"})
        plt.title(f"Geographische Daten mit Attributen für Tabelle '{table_name}'")
        plt.show()

        # Weitere Analysen
        # Beispiel: Durchschnittswert pro geographische Einheit berechnen
        average_values = merged.groupby('geo_column')['data_column'].mean().reset_index()
        print(f"Durchschnittswerte pro geographische Einheit in Tabelle '{table_name}':")
        print(average_values)

        # Visualisierung der Durchschnittswerte auf der Karte
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        merged_avg = gdf.merge(average_values, left_on='id', right_on='geo_column')
        merged_avg.plot(column='data_column', ax=ax, legend=True,
                        legend_kwds={'label': "Average Value",
                                     'orientation': "horizontal"})
        plt.title(f"Durchschnittswerte pro geographische Einheit in Tabelle '{table_name}'")
        plt.show()
    else:
        print(f"Tabelle '{table_name}' enthält keine 'data_column' Spalte für die Visualisierung.")
        
print("Geographische Datenanalyse abgeschlossen.")

  


Geographische Daten:
   id    name                                           geometry
0   1  Unit 1  POLYGON ((-99.00000 41.00000, -99.00000 42.000...
1   2  Unit 2  POLYGON ((-100.00000 43.00000, -100.00000 44.0...

Analysiere Tabelle: Top 50 – Argentinien 2024-05-18
Tabelle 'Top 50 – Argentinien 2024-05-18' enthält keine 'location_id' Spalte. Überspringen...

Analysiere Tabelle: Top 50 – Australien 2024-05-18
Tabelle 'Top 50 – Australien 2024-05-18' enthält keine 'location_id' Spalte. Überspringen...

Analysiere Tabelle: Top 50 – Belarus 2024-05-18
Tabelle 'Top 50 – Belarus 2024-05-18' enthält keine 'location_id' Spalte. Überspringen...

Analysiere Tabelle: Top 50 – Belgien 2024-05-18
Tabelle 'Top 50 – Belgien 2024-05-18' enthält keine 'location_id' Spalte. Überspringen...

Analysiere Tabelle: Top 50 – Bolivien 2024-05-18
Tabelle 'Top 50 – Bolivien 2024-05-18' enthält keine 'location_id' Spalte. Überspringen...

Analysiere Tabelle: Top 50 – Brasilien 2024-05-18
Tabelle 'Top 50 – Bras

# Teil 5: Fortgeschrittene Deep Learning Techniken

In [ ]:
#Erforderliches Pakete installieren:
pip install tensorflow pandas sqlalchemy pymysql nltk

In [17]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sqlalchemy import create_engine, inspect
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Sicherstellen, dass die NLTK-Ressourcen heruntergeladen sind
nltk.download('vader_lexicon')

# Sensible Daten aus Umgebungsvariablen lesen
db_user = os.getenv('DB_USER', 'root')
db_password = os.getenv('DB_PASSWORD', 'example')
db_host = os.getenv('DB_HOST', 'localhost')
db_name = os.getenv('DB_NAME', 'spotify_data')

# Verbindung zur MySQL-Datenbank herstellen
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Inspector verwenden, um Tabelleninformationen abzurufen
inspector = inspect(engine)

# Alle Tabellennamen abrufen
table_names = inspector.get_table_names()

# Funktion zur Bewertung und Visualisierung des Modells
def evaluate_and_visualize_model(model, X_test_scaled, y_test, history, model_name):
    y_pred = model.predict(X_test_scaled)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} - Mean Squared Error: {mse}")
    print(f"{model_name} - R² Score: {r2}")

    # Trainings- und Validierungsverluste visualisieren
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Training and Validation Loss - {model_name}')
    plt.show()

    # Visualisierung der tatsächlichen vs. vorhergesagten Werte
    plt.figure(figsize=(10, 5))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title(f'True vs Predicted Values - {model_name}')
    plt.show()

# Durchlaufen aller Tabellen
for table_name in table_names:
    print(f"\nAnalysiere Tabelle: {table_name}")
    query = f"SELECT * FROM `{table_name}`"
    df = pd.read_sql(query, engine)
    
    if df.shape[1] < 2:
        print(f"Nicht genug Daten in Tabelle '{table_name}' für ML-Analyse.")
        continue
    
    print(f"Erster Blick auf die Daten in Tabelle '{table_name}':")
    print(df.head())
    print("\n")
    
    # Datenvorverarbeitung
    print("Datenvorverarbeitung...")

    # Annahme: Die letzte Spalte ist das Ziel (Target) und die restlichen Spalten sind Features
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Train-Test-Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Skalierung der Daten
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Modell erstellen - MLP
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)  # Regression - keine Aktivierungsfunktion in der Ausgangsschicht
    ])

    # Modell kompilieren
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

    # Modelltraining mit Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train_scaled, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )

    # Modellbewertung und Visualisierung - MLP
    print("Modellbewertung - MLP...")
    evaluate_and_visualize_model(model, X_test_scaled, y_test, history, "MLP")

    # Ergänzung: Verwendung eines Convolutional Neural Networks (CNN)

    # Reshape der Daten für CNN
    X_train_cnn = X_train_scaled.reshape(-1, X_train_scaled.shape[1], 1, 1)
    X_test_cnn = X_test_scaled.reshape(-1, X_test_scaled.shape[1], 1, 1)

    # Modell erstellen - CNN
    cnn_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(X_train_cnn.shape[1], 1, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)  # Regression - keine Aktivierungsfunktion in der Ausgangsschicht
    ])

    # Modell kompilieren - CNN
    cnn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

    # Modelltraining mit Early Stopping - CNN
    history_cnn = cnn_model.fit(
        X_train_cnn, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )

    # Modellbewertung und Visualisierung - CNN
    print("Modellbewertung - CNN...")
    evaluate_and_visualize_model(cnn_model, X_test_cnn, y_test, history_cnn, "CNN")

    # Ergänzung: Verwendung von NLP-Techniken (z.B. Sentiment Analysis)

    # Sentiment Analysis mit VADER
    sia = SentimentIntensityAnalyzer()

    # Optional: Sentiment Analysis auf Textdaten in der Datenbank anwenden
    # Angenommen, es gibt eine Spalte 'TextColumn' in der Datenbank
    if 'TextColumn' in df.columns:
        df['sentiment'] = df['TextColumn'].apply(lambda x: sia.polarity_scores(x)['compound'])
        print("Sentiment Analysis auf Textdaten angewendet.")
        print(df[['TextColumn', 'sentiment']].head())



Analysiere Tabelle: Top 50 – Argentinien 2024-05-18
Erster Blick auf die Daten in Tabelle 'Top 50 – Argentinien 2024-05-18':
   Unnamed: 0                Track_ID  Artist_Name  \
0           0  3GD6eImRvT0zgr8cQnokUq        Bhavi   
1           1  5rQSQlZXXjMcevPGoAfE1z  Salastkbron   
2           2  4wS0TnQzVkY9ML1BPKpOk1    Tiago PZK   
3           3  7bywjHOc0wSjGGbj04XbVi         Feid   
4           4  6XjDF6nds4DE2BBbagZol6   FloyyMenor   

                                          Track_Name  Popularity Explicit  \
0  BESAME (feat. Tiago PZK, Khea & Neo Pistea) - ...          86        1   
1                                      Un Besito Más          79        0   
2                                               Piel          86        1   
3                                               LUNA          93        0   
4                                          Gata Only          98        1   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/bavaarde/nltk_data...


ValueError: could not convert string to float: '0mJRX0WnqWlu8Boe7gpQ1P'

# Teil 6: Modellinterpretation und Evaluation

In [ ]:
# Erforderliche Pakete installieren:
pip install tensorflow pandas sqlalchemy pymysql scikit-learn matplotlib

In [18]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sqlalchemy import create_engine, inspect
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Sensible Daten aus Umgebungsvariablen lesen
db_user = os.getenv('DB_USER', 'root')
db_password = os.getenv('DB_PASSWORD', 'example')
db_host = os.getenv('DB_HOST', 'localhost')
db_name = os.getenv('DB_NAME', 'spotify_data')

# Verbindung zur MySQL-Datenbank herstellen
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Inspector verwenden, um Tabelleninformationen abzurufen
inspector = inspect(engine)

# Alle Tabellennamen abrufen
table_names = inspector.get_table_names()

# Funktion zur Modellbewertung und Visualisierung
def evaluate_model(model, X_test_scaled, y_test, y_pred_proba, history):
    y_pred = (y_pred_proba > 0.5).astype("int32")

    # Berechnung der Modellgütemaße
    mse = mean_squared_error(y_test, y_pred_proba)
    r2 = r2_score(y_test, y_pred_proba)
    print(f"Mean Squared Error: {mse}")
    print(f"R² Score: {r2}")

    # Zusätzliche Modellgütemaße für Klassifikation
    accuracy = np.mean(y_pred == y_test)
    print(f"Accuracy: {accuracy}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # ROC und AUC
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    print(f"ROC AUC Score: {roc_auc}")

    # Visualisierung der Trainings- und Validierungsverluste
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()

    # Visualisierung der tatsächlichen vs. vorhergesagten Werte
    plt.figure(figsize=(10, 5))
    plt.scatter(y_test, y_pred_proba, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
    plt.xlabel('True Values')
    plt.ylabel('Predicted Probabilities')
    plt.title('True vs Predicted Probabilities')
    plt.show()

    # ROC Kurve visualisieren
    plt.figure(figsize=(10, 5))
    plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')
    plt.show()

# Durchlaufen aller Tabellen
for table_name in table_names:
    print(f"\nAnalysiere Tabelle: {table_name}")
    query = f"SELECT * FROM `{table_name}`"
    df = pd.read_sql(query, engine)
    
    if df.shape[1] < 2:
        print(f"Nicht genug Daten in Tabelle '{table_name}' für ML-Analyse.")
        continue
    
    print(f"Erster Blick auf die Daten in Tabelle '{table_name}':")
    print(df.head())
    print("\n")
    
    # Datenaufbereitung
    print("Datenvorverarbeitung...")

    # Annahme: Die letzte Spalte ist das Ziel (Target) und die restlichen Spalten sind Features
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # Train-Test-Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Skalierung der Daten
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Modell erstellen
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  # Sigmoid-Aktivierung für binäre Klassifikation
    ])

    # Modell kompilieren
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Modelltraining mit Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train_scaled, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )

    # Modellbewertung
    print("Modellbewertung...")
    y_pred_proba = model.predict(X_test_scaled)
    evaluate_model(model, X_test_scaled, y_test, y_pred_proba, history)

print("Modellinterpretation und Evaluation abgeschlossen.")



Analysiere Tabelle: Top 50 – Argentinien 2024-05-18
Erster Blick auf die Daten in Tabelle 'Top 50 – Argentinien 2024-05-18':
   Unnamed: 0                Track_ID  Artist_Name  \
0           0  3GD6eImRvT0zgr8cQnokUq        Bhavi   
1           1  5rQSQlZXXjMcevPGoAfE1z  Salastkbron   
2           2  4wS0TnQzVkY9ML1BPKpOk1    Tiago PZK   
3           3  7bywjHOc0wSjGGbj04XbVi         Feid   
4           4  6XjDF6nds4DE2BBbagZol6   FloyyMenor   

                                          Track_Name  Popularity Explicit  \
0  BESAME (feat. Tiago PZK, Khea & Neo Pistea) - ...          86        1   
1                                      Un Besito Más          79        0   
2                                               Piel          86        1   
3                                               LUNA          93        0   
4                                          Gata Only          98        1   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0

ValueError: could not convert string to float: '0mJRX0WnqWlu8Boe7gpQ1P'